### Connect to workspace

In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /home/demouser/repos/HighPerformanceTensorFlowOnAzure/aml_config/config.json
jkaml
jkaml
eastus2
952a710c-8d9c-40c1-9fec-f752138cc0b3


### Retrieve the model

In [2]:
from azureml.core import Experiment

exp = Experiment(ws, name='aerial-classifier-train')
tags = {"FriendlyName": "FirstRun"}
runs = exp.get_runs(tags=tags)
try:
    run = next(runs)
except:
    print("Could not find the run with {}".format(tags))
    
run.download_file('outputs/aerial_classifier.hd5', 'aerial_classifier.hd5')

### Load the model

In [3]:
import tensorflow.keras as keras
top_model = keras.models.load_model('aerial_classifier.hd5')
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 1,052,166
Trainable params: 1,052,166
Non-trainable params: 0
_________________________________________________________________


### Assemble the model
Stack the *top_model* on top of **ResNet50** "trunk"

In [7]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras import Model

# Create a featurizer
trunk = resnet50.ResNet50(
                weights = 'imagenet', 
                input_shape=(224,224,3), 
                include_top = False,
                pooling = 'avg')

full_model = Model(inputs=trunk.input, outputs=top_model(trunk.output))
full_model.summary()

### Create a dataset

In [ ]:
# Returns a dataset based on a list of TFRecords files passsed as a parameters. 
def create_dataset(files, batch_size=32, prefetch_buffer_size=1, train=True, buffer_size=10000):
    IMAGE_SHAPE = (224, 224, 3,)
    NUM_CLASSES = 6
          
    # Extract image and label from proto Example
    def _parse(example_proto):
        features = {'label': tf.FixedLenFeature((), tf.int64, default_value=0),
                    'image': tf.FixedLenFeature((), tf.string, default_value="")}
        parsed_features = tf.parse_single_example(example_proto, features)
        label = parsed_features['label']
        label = tf.one_hot(label, NUM_CLASSES)
        image = image = tf.decode_raw(parsed_features['image'], tf.uint8)
        image = tf.cast(image, tf.float32)
        image = tf.reshape(image, IMAGE_SHAPE)
                                                                  
        # Pre-process image data for ResNet
        #   RGB -> BGR
        image = image[..., ::-1]
        #   Substract the Imagenet mean for each channel
        imagenet_mean=tf.constant(-np.array([103.939, 116.779, 123.68]), dtype=tf.float32)
        image = tf.nn.bias_add(image, imagenet_mean)
        #image = tf.subtract(image, imagenet_mean)
        #image = resnet50.preprocess_input(image)
        
        return image, label

    dataset = tf.data.TFRecordDataset(files)
    dataset = dataset.map(_parse, num_parallel_calls=FLAGS.num_parallel_calls)
    #dataset = dataset.map(_parse)
    if train:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=prefetch_buffer_size)
    dataset = dataset.repeat()

    return dataset